## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [9]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice
import networkx as nx

def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [113]:
def weight_tree(pot, vertex_coordinates, T):
    result = 0
    for i in range(len(T)):
        result += pot[T[i][0]] + pot[T[i][1]] + euclidean_distance(vertex_coordinates[T[i][0]], vertex_coordinates[T[i][1]])

    return result

def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    n = len(vertex_coordinates)
    
    alp = 35
    pot = [0] * n
    sum_pot = 0
    
    result = 0
    
    for it in range(20):
        G = nx.Graph()
        G.add_nodes_from(range(0, n))
        for i in range(n):
            for j in range(i + 1, n):
                G.add_edge(i, j, weight=(euclidean_distance(vertex_coordinates[i], vertex_coordinates[j]) + pot[i] + pot[j]))
        T = list(nx.minimum_spanning_edges(G, data=False))
        
        cnt = [0] * n
        
        for i in range(len(T)):
            cnt[T[i][0]] += 1
            cnt[T[i][1]] += 1
        
        result = max(result, weight_tree(pot, vertex_coordinates, T) - 2 * sum_pot)
        #print(weight_tree(pot, vertex_coordinates, T) - 2 * sum_pot)
        
        minpot = 0
        
        for i in range(n):
            delta = cnt[i] - 2
            
            cf = alp / (it + 1) * delta
            pot[i] += cf
            minpot = min(minpot, pot[i])
            sum_pot += cf
    
        sum_pot -= n * minpot
        
        for i in range(n):
            pot[i] -= minpot
    
    return result

In [115]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [116]:
run_all()

Instance d198.tsp… done in 7.1 seconds with lower bound 12379
Instance d493.tsp… done in 4.9e+01 seconds with lower bound 32017
Instance d657.tsp… done in 9e+01 seconds with lower bound 46403
Instance d2103.tsp… done in 1.1e+03 seconds with lower bound 76892
Instance pr107.tsp… done in 1.8 seconds with lower bound 36994
Instance pr152.tsp… done in 3.9 seconds with lower bound 61243
Instance pr439.tsp… done in 3.7e+01 seconds with lower bound 97041


Результаты верхних оценок:

Solving instance d198.tsp… done in 0.22 seconds with tour length 18620 using NN and in 3.6 seconds with tour length 18052 using NI

Solving instance d493.tsp… done in 1.3 seconds with tour length 43646 using NN and in 5.9e+01 seconds with tour length 41576 using NI

Solving instance d657.tsp… done in 2.4 seconds with tour length 62176 using NN and in 1.4e+02 seconds with tour length 60195 using NI

Solving instance d2103.tsp… done in 2.3e+01 seconds with tour length 87468 using NN and in 4.7e+03 seconds with tour length 86027 using NI

Solving instance pr107.tsp… done in 0.064 seconds with tour length 46678 using NN and in 0.66 seconds with tour length 53211 using NI

Solving instance pr152.tsp… done in 0.13 seconds with tour length 85702 using NN and in 1.8 seconds with tour length 86914 using NI

Solving instance pr439.tsp… done in 1.1 seconds with tour length 131282 using NN and in 4.1e+01 seconds with tour length 132780 using NI


## Выводы
Как мы видим, результаты верхних оценок всегда не более чем в 1.5 раза хуже чем нижних, иногда это отношение даже достигает значения 1.13, что показывает что алгоритмы NI и NN работают достаточно хорошо для решения задачи TSP.